In [ ]:
!pip install psycopg2

In [1]:
import psycopg2
import sys, os
import numpy as np
import pandas as pd
import creds

In [4]:
# Set up a connection to the postgres server.
conn_string = "host="+ creds.PGHOST +" port="+ "5432" +" dbname="+ creds.PGDATABASE +" user=" + creds.PGUSER \
+" password="+ creds.PGPASSWORD
conn=psycopg2.connect(conn_string)
print("Connected!")

# Create a cursor object
cursor = conn.cursor()

Connected!


In [5]:
cursor.execute("""SELECT table_name FROM information_schema.tables
       WHERE table_schema = 'public'""")
for table in cursor.fetchall():
    print(table)

('Banner',)
('Beacon',)
('CandiePrize',)
('BoosterPromotion',)
('BeaconLog',)
('DrawBidTransaction',)
('CandieAd',)
('CandieAdCoupon',)
('CandieAdImpression',)
('CandieSpin',)
('CandieTransactionView',)
('SurpriseCandie',)
('DrawPrizeBucket',)
('CandieTransactionArchive',)
('User',)
('WalletView',)
('CandieTransactionSettlement',)
('Coupon',)
('DrawPeriod',)
('DrawPrize',)
('GIROHistory',)
('Config',)
('Achievement',)
('AcquisitionChannel',)
('Badge',)
('Bank',)
('GlobalNotification',)
('InvoicePatch',)
('Notification',)
('NonPartnerMerchant',)
('Organization',)
('NonPartnerMerchant_MerchantStoreCategory',)
('MerchantShugaRate',)
('Merchant_MerchantStoreCategory',)
('Merchant',)
('Product',)
('ProductCategory',)
('Profile',)
('ReceiptImage',)
('Purchase',)
('Promotion',)
('Receipt',)
('ReceiptProduct',)
('ReceiptProduct_ProductCategory',)
('RedeemProduct',)
('RedeemProduct_ProductCategory',)
('RedeemTransaction',)
('Rule',)
('Store_Product',)
('knex_migrations',)
('ShugaTransactionView

In [6]:
#Get Table columns for Receipts table

cursor.execute('Select * FROM "Receipt" LIMIT 0')
colnames = [desc[0] for desc in cursor.description]
colnames

['id',
 'createdAt',
 'updatedAt',
 'purchaseDate',
 'total',
 'invoiceNo',
 'isReviewed',
 'isGranted',
 'isRejected',
 'comment',
 'store_id_fk',
 'user_id_fk',
 'shugaTransaction_id_fk',
 'candieTransaction_id_fk',
 'merchantName',
 'address',
 'parsedText',
 'isSystemRejected',
 'isUserReport',
 'merchantRegistrationId',
 'merchant_id_fk',
 'nonPartnerMerchant_id_fk',
 'shouldGrantAt',
 'exceptionName',
 'opsComment',
 'reviewedBy',
 'dataQualityScore',
 'userReportType',
 'activeBadges',
 'purchaseRawDate',
 'purchaseRawTime']

In [7]:
#Get Table columns for MerchantStoreCategory table

cursor.execute('Select * FROM "MerchantStoreCategory" LIMIT 0')
colnames = [desc[0] for desc in cursor.description]
colnames

['id', 'createdAt', 'updatedAt', 'name', 'description', 'industry']

In [8]:
#Get Table columns for Merchant_MerchantStoreCategory table

cursor.execute('Select * FROM "Merchant_MerchantStoreCategory" LIMIT 0')
colnames = [desc[0] for desc in cursor.description]
colnames

['id', 'createdAt', 'merchant_id_fk', 'merchantStoreCategory_id_fk']

In [9]:
#Get Table columns for NonPartnerMerchant_MerchantStoreCategory table

cursor.execute('Select * FROM "NonPartnerMerchant_MerchantStoreCategory" LIMIT 0')
colnames = [desc[0] for desc in cursor.description]
colnames

['id', 'createdAt', 'nonPartnerMerchant_id_fk', 'merchantStoreCategory_id_fk']

In [10]:
#Get Table columns for User table

cursor.execute('Select * FROM "User" LIMIT 0')
colnames = [desc[0] for desc in cursor.description]
colnames

['id',
 'createdAt',
 'updatedAt',
 'email',
 'facebookId',
 'pushToken',
 'mobile',
 'smsSent',
 'password',
 'shuga',
 'referrer',
 'candie',
 'bonusShuga',
 'isEnabledPushNotification',
 'isEnabledMarketingMessage',
 'platform',
 'acquisitionChannel_id_fk',
 'isEmailVerified',
 'originalEmail',
 'isEmailBounced',
 'isReferralBonusGranted',
 'candieClick']

In [11]:
# Build SQL statement retriving single joined table

sql = \
'\
SELECT "NonPartnerMerchant_MerchantStoreCategory"."merchantStoreCategory_id_fk" AS "merchantStoreCategory_id_fk_x", "Merchant_MerchantStoreCategory"."merchantStoreCategory_id_fk" AS "merchantStoreCategory_id_fk_y", "Receipt".id, "Receipt"."total", "Receipt"."user_id_fk", "Receipt"."merchant_id_fk", "Receipt"."nonPartnerMerchant_id_fk" FROM "Receipt" \
LEFT JOIN "Merchant_MerchantStoreCategory" on "Receipt"."merchant_id_fk" = "Merchant_MerchantStoreCategory"."merchant_id_fk" \
LEFT JOIN "NonPartnerMerchant_MerchantStoreCategory" on "Receipt"."nonPartnerMerchant_id_fk" = "NonPartnerMerchant_MerchantStoreCategory"."nonPartnerMerchant_id_fk"\
WHERE "isGranted" = TRUE \
ORDER BY id \
'

receipts_notclean = pd.read_sql(sql, conn, index_col='id')
storecategory = pd.read_sql('SELECT id AS "merchantStoreCategory_id_fk", "industry" FROM "MerchantStoreCategory" ORDER BY id', conn)

In [12]:
receipts_notclean['merchantStoreCategory_id_fk'] = receipts_notclean[['merchantStoreCategory_id_fk_x','merchantStoreCategory_id_fk_y']].max(axis=1)
receipts = receipts_notclean.dropna(subset=['merchantStoreCategory_id_fk'])
receipts['merchantStoreCategory_id_fk'] = receipts['merchantStoreCategory_id_fk'].astype(int)

C:\Users\Ian\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [ ]:
df = receipts.merge(storecategory,on='merchantStoreCategory_id_fk', how = 'left')
df.drop(['merchant_id_fk','merchantStoreCategory_id_fk','merchantStoreCategory_id_fk_x','merchantStoreCategory_id_fk_y','nonPartnerMerchant_id_fk'], axis=1, inplace=True)
df

,total,user_id_fk,industry
0,6.70,3,Food
1,4.80,3,Food
2,58.00,8,Food
3,39.70,7,Supermarket
4,9.00,6,Food
5,32.80,10,Health and Beauty
6,37.20,10,Health and Beauty
7,41.80,10,Health and Beauty
8,60.00,6,Food
9,26.90,3,Food


In [ ]:
df = pd.concat([df, pd.get_dummies(df['industry'])], axis=1)
df

,total,user_id_fk,industry,Automobile,"Baby, Kids & Toys",Book & Stationery,Convenience Store,Departmental Store,Electronics,Fashion,Food,Health and Beauty,Health and Fitness,Hotels & Accommodation,Household,Lifestyle,Services,Supermarket,Transportation,Utilities
0,6.70,3,Food,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0
1,4.80,3,Food,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0
2,58.00,8,Food,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0
3,39.70,7,Supermarket,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0
4,9.00,6,Food,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0
5,32.80,10,Health and Beauty,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0
6,37.20,10,Health and Beauty,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0
7,41.80,10,Health and Beauty,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0
8,60.00,6,Food,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0
9,26.90,3,Food,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0


In [ ]:
userdata = df.groupby(['user_id_fk']).sum()
userdata

,total,Automobile,"Baby, Kids & Toys",Book & Stationery,Convenience Store,Departmental Store,Electronics,Fashion,Food,Health and Beauty,Health and Fitness,Hotels & Accommodation,Household,Lifestyle,Services,Supermarket,Transportation,Utilities
user_id_fk,,,,,,,,,,,,,,,,,,
3,606.31,0.0,0.0,0.0,0.0,0.0,0.0,0.0,11.0,1.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0
6,1351.47,0.0,1.0,0.0,0.0,1.0,0.0,4.0,12.0,1.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0
7,7748.36,0.0,0.0,3.0,1.0,4.0,5.0,7.0,104.0,12.0,0.0,0.0,5.0,2.0,3.0,35.0,0.0,0.0
8,12187.41,0.0,0.0,1.0,5.0,2.0,1.0,9.0,48.0,4.0,0.0,0.0,5.0,1.0,12.0,7.0,1.0,0.0
9,633.02,0.0,0.0,1.0,1.0,1.0,0.0,1.0,12.0,1.0,0.0,0.0,0.0,0.0,0.0,8.0,0.0,0.0
10,14375.95,0.0,0.0,5.0,1.0,1.0,2.0,7.0,63.0,8.0,0.0,0.0,5.0,0.0,1.0,16.0,0.0,0.0
18,17.40,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
21,6.80,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
24,19.20,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
userdata_nototal = userdata.drop(['total'], axis=1)
userdata_nototal

,Automobile,"Baby, Kids & Toys",Book & Stationery,Convenience Store,Departmental Store,Electronics,Fashion,Food,Health and Beauty,Health and Fitness,Hotels & Accommodation,Household,Lifestyle,Services,Supermarket,Transportation,Utilities
user_id_fk,,,,,,,,,,,,,,,,,
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,11.0,1.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0
6,0.0,1.0,0.0,0.0,1.0,0.0,4.0,12.0,1.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0
7,0.0,0.0,3.0,1.0,4.0,5.0,7.0,104.0,12.0,0.0,0.0,5.0,2.0,3.0,35.0,0.0,0.0
8,0.0,0.0,1.0,5.0,2.0,1.0,9.0,48.0,4.0,0.0,0.0,5.0,1.0,12.0,7.0,1.0,0.0
9,0.0,0.0,1.0,1.0,1.0,0.0,1.0,12.0,1.0,0.0,0.0,0.0,0.0,0.0,8.0,0.0,0.0
10,0.0,0.0,5.0,1.0,1.0,2.0,7.0,63.0,8.0,0.0,0.0,5.0,0.0,1.0,16.0,0.0,0.0
18,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
21,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
24,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
from sklearn.cluster import KMeans

In [ ]:
# Elbow method

Distortion = []
K = range(1,15)
for k in K:
    km = KMeans(n_clusters=k)
    km = km.fit(userdata_nototal)
    Distortion.append(km.inertia_)
    

In [ ]:
import matplotlib.pyplot as plt
plt.plot(K, Distortion, 'bx-')
plt.xlabel('k')
plt.ylabel('Distortion')
plt.title('Elbow Method For Optimal k')
plt.show()

In [ ]:
# set number of clusters
kclusters = 4

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(userdata_nototal)

# add clustering labels
userdata.insert(0, 'Cluster Labels', kmeans.labels_)

userdata.head()

In [ ]:
userdata.loc[userdata['Cluster Labels'] == 0]

In [ ]:
userdata.loc[userdata['Cluster Labels'] == 1]

In [ ]:
userdata.loc[userdata['Cluster Labels'] == 2]

In [ ]:
userdata.loc[userdata['Cluster Labels'] == 3]

In [ ]:
import seaborn as sns

userdatasns = pd.melt(userdata.drop(['total'], axis=1), id_vars='Cluster Labels', var_name='Type', value_name='number')

plt.rcParams['figure.figsize'] = [20, 5]

sns.set(style='whitegrid')
ax = sns.catplot(x='Cluster Labels', y='number', hue='Type', data=userdatasns, kind='bar')
ax.set(xlabel='Cluster Labels', ylabel='Number')

ax.savefig('output.svg', format='svg', dpi=1200)
